In [1]:
import pandas as pd
import numpy as np
import folium
import json

import io
from PIL import Image

import seaborn as sns

import sys

import matplotlib.pyplot as plt

from folium import plugins
from folium.plugins import MeasureControl # Adds tool to the top right
from folium.plugins import FloatImage # Fairly obvious I imagine - works best with transparent backgrounds
from folium.plugins import HeatMap

# Geopandas can be used to load the .shp file of Illes Balears.
import geopandas as gpd

In [4]:
data_folder = '~/Documents/Other/TerraCremada/data/'
datafile_tourism = 'TurismeMallorca.csv'
datafile_population = 'population_ine.csv'
datafile_surface = 'superficie_ibestat.csv'
datafile_renta = 'renta_ine.csv'
datafile_geo = 'geo_details.csv'

datapath_tourism = "{}{}".format(data_folder, datafile_tourism)
datapath_population = "{}{}".format(data_folder, datafile_population)
datapath_surface = "{}{}".format(data_folder, datafile_surface)
datapath_renta = "{}{}".format(data_folder, datafile_renta)
datapath_geo = "{}{}".format(data_folder, datafile_geo)

df1 = pd.read_csv(datapath_tourism, sep = ',', index_col=None, header=0)
df1 = df1.set_index('Municipi')
df1['pc_total'] = df1.pc_hotel + df1.pc_llvac
df1 = df1.sort_values(by='pc_total', axis = 0, ascending = False)
df1 = df1.reset_index().rename(columns={'Municipi': 'mun_name',
                                        'pc_hotel': 'pc_hotel',
                                        'pc_llvac': 'pc_llvac',
                                        'pc_total': 'pc_total'})

df2 = pd.read_csv(datapath_population, sep = ',', index_col=None, header=0, thousands='.')
df2 = df2.drop(['Municipios','year'], axis = 1)
#df3 = df3.set_index('mun_name')
df2 = df2.pivot(columns='gender', index='mun_name', values='population')
df2 = df2.reset_index()
df2 = df2.rename(columns={'mun_name': 'mun_name','Total': 'pop_total'})[['mun_name', 'pop_total']]

# Superficie esta en hecaterees per defecte
df3 = pd.read_csv(datapath_surface, sep = '\t', index_col=None, header=1, thousands='.', decimal=',')
df3['sup_m2'] =df3.superficie*10000 # 1 ha = 10000 m2

df4 = pd.read_csv(datapath_renta, sep = '\t', index_col=None, header=0, thousands='.', decimal=',')
df4 = df4.pivot(columns='Indicadores de renta media y mediana', index='mun_name', values='Total')
df4 = df4.reset_index()
df4 = df4.rename(columns={'mun_name': 'mun_name',
                          'Renta neta media por persona ': 'inc_net_pp',
                          'Renta neta media por hogar': 'inc_net_ph',
                          'Media de la renta por unidad de consumo': 'mean_inc_consunit',
                          'Mediana de la renta por unidad de consumo': 'median_inc_consunit',
                          'Renta bruta media por persona': 'inc_gross_pp',
                          'Renta bruta media por hogar': 'inc_gross_ph'})

df5 = pd.read_csv(datapath_geo, sep = ',', index_col=None, header=0, thousands='.', decimal=',')

data = pd.merge(df1, df2, on=["mun_name"])
data = pd.merge(data, df3, on=["mun_name"])
data = pd.merge(data, df4, on=["mun_name"])
data = pd.merge(data, df5, on=["mun_name"])

# Vacancy Per 100 people
data['vp100p'] = data['pc_total']*100/data['pop_total']

data['pc_hotel_p100p'] = data['pc_hotel']*100/data['pop_total']
data['pc_llvac_p100p'] = data['pc_llvac']*100/data['pop_total']

# Vacancies per squared kilometer
data['pc_hotel_pkm2'] = data['pc_hotel']*100/data['superficie']
data['pc_llvac_pkm2'] = data['pc_llvac']*100/data['superficie']

data['ratio_population'] = data['pop_total']*100/data['superficie']

data['diff_pc'] = data['pop_total'] - data['pc_total']

# Selecting the 3/4 of the municipalities with more population
population_quartiles = data['pop_total'].quantile([0,0.25,0.5,0.75,1]).tolist()
data['large_pop'] =  data['pop_total'] > population_quartiles[1]

# Rename the section with region name 
mallorca_regions = {1:'Llevant', 2: 'Migjorn', 3:'Palma', 4:'Pla', 5:'Raiguer', 6:'Tramuntana'}
data['region_name'] = data.region.map(mallorca_regions).astype("category")

coast_regions = {0:'Interior', 1: 'Costa'}
data['coast'] = data.coast.map(coast_regions).astype('category')

# The study focuses on Mallorca, so we discard other IB municipalities
mun_names_IB = ['Estellencs', 'Formentera', 'EsMercadal', 'Inca', 'Mancor', 'StJosepTalaia','StLlorenc',
                'StLluis', 'Bunyola', 'Petra', 'StJoanLabritja', 'SesSalines', 'Algaida', 'Campos', 'SaPobla',
                'Capdepera', 'StaEulalia', 'Escorca', 'Porreres', 'StJoan', 'Alaro', 'Ferreries', 'Lloret',
                'Maria', 'StaEugenia', 'Buger', 'Lloseta', 'EsCastell', 'Binissalem', 'Deia', 'Mao', 'Marratxi',
                'Selva', 'Palma', 'Manacor', 'Montuiri', 'StaMaria', 'Alcudia', 'Andratx', 'Santanyi',
                'Llucmajor', 'Vilafranca', 'Alaior', 'Consell','Sineu', 'Calvia', 'Campanet', 'Llubi',
                'StAntoniPortmany', 'SonServera', 'Ariany', 'Banyalbufar', 'StaMargalida', 'Soller',
                'Valldemossa', 'Arta', 'Costitx', 'Ciutadella', 'Esporles', 'Felanitx', 'Eivissa', 'Muro',
                'Pollenca', 'Puigpunyent', 'Sencelles', 'Fornalutx', 'EsMigjorn'] 

mun_names_mal = data.reset_index().mun_name.tolist()
shared_names = list(set(mun_names_mal) & set(mun_names_IB))

# Load the .shp file
shpfile_name = 'georef-spain-municipio-millesime.shp'
shpfile_folder = 'georef-spain-municipio/'
IBmap_gdf = gpd.read_file("{}{}".format(data_folder,shpfile_folder,shpfile_name))

# Correcting the name to avoid "unknown" characters
IBmap_gdf['mun_name0'] = IBmap_gdf['mun_name']
IBmap_gdf['mun_name'] = mun_names_IB

# Checking that the lengths are correct
#print(len(shared_names), len(mun_names_mal))

Malmap_gdf = IBmap_gdf[IBmap_gdf.mun_name.isin(shared_names)]

right = Malmap_gdf[['year', 'mun_code', 'mun_name','geometry']]
gdf1 = pd.merge(data, right, on=["mun_name"])

# Need to convert into a json file (which is a string)
gdf1_json = gpd.GeoDataFrame(gdf1).to_json()

---------------------------------

In [ ]:
# https://coolors.co/palette/001524-15616d-ffecd1-ff7d00-78290f
background_color = '#FFF5E8'
color1 = '#ff7d00'
color2 = '#15616d'
color3 = '#78290f'
color31 = '#D6491A' # faded color 3